# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1449883536 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/Alex/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1449883536 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to akorotcov@yahoo.com and will expire on September 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-37935 - Server binary: /Users/Alex/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1449883536.log
[INFO] GraphLab Server Version: 1.7.1


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
train_data,test_data = sales.random_split(.8,seed=0)

In [4]:
train_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: Number of coefficients    : 4
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 1.030898     | 4146407.600631     | 258679.804477 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [6]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | 87910.0724924  |
| sqft_living |  None | 315.403440552  |
|   bedrooms  |  None | -65080.2155528 |
|  bathrooms  |  None | 6944.02019265  |
+-------------+-------+----------------+
[4 rows x 3 columns]



# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [7]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

271789.505878


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [10]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    prediction_model = model.predict(data)
    # Then compute the residuals/errors
    errors = outcome - prediction_model
    # Then square and add them up
    RSS = sum(errors**2)
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [11]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.7376153833e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [12]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [14]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [15]:
train_data['bedrooms', 'bathrooms']

bedrooms,bathrooms
3.0,1.0
3.0,2.25
2.0,1.0
4.0,3.0
3.0,2.0
4.0,4.5
3.0,2.25
3.0,1.5
3.0,1.0
3.0,2.5


In [24]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))
train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

In [25]:
test_data

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980,2,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553,2,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186,1,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495,2,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400,1,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807,1.5,0


* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [27]:
mean_bedrooms_squared_test = test_data['bedrooms_squared'].mean()
print "Mean bedrooms_squared = ", mean_bedrooms_squared_test 

mean_bed_bath_rooms_test = test_data['bed_bath_rooms'].mean()
print "Mean bed_bath_rooms = ", mean_bed_bath_rooms_test

mean_log_sqft_living_test = test_data['log_sqft_living'].mean()
print "Mean log_sqft_living = ", mean_log_sqft_living_test

mean_lat_plus_long_test = test_data['lat_plus_long'].mean()
print "Mean lat_plus_long = ", mean_lat_plus_long_test 

Mean bedrooms_squared =  12.4466777016
Mean bed_bath_rooms =  7.50390163159
Mean log_sqft_living =  7.55027467965
Mean lat_plus_long =  -74.6533349722


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [28]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [31]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features,
                                                  validation_set = None)
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features,
                                                  validation_set = None)
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features,
                                                  validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROGRESS: Number of features          : 5
PROGRESS: Number of unpacked features : 5
PROGRESS: Number of coefficients    : 6
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.052000     | 4074878.213096     | 236378.596455 |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:
PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 17384
PROG

In [33]:
# Examine/extract each model's coefficients:
model_1_weight_summary = model_1.get("coefficients")
print "model_1 weight_summary"
print model_1_weight_summary

model_2_weight_summary = model_2.get("coefficients")
print "model_2 weight summary"
print model_2_weight_summary

model_3_weight_summary = model_3.get("coefficients")
print "model_3 weight summary"
print model_3_weight_summary

 model_1 weight_summary
+-------------+-------+----------------+
|     name    | index |     value      |
+-------------+-------+----------------+
| (intercept) |  None | -56140675.7444 |
| sqft_living |  None | 310.263325778  |
|   bedrooms  |  None | -59577.1160682 |
|  bathrooms  |  None | 13811.8405418  |
|     lat     |  None | 629865.789485  |
|     long    |  None | -214790.285186 |
+-------------+-------+----------------+
[6 rows x 3 columns]

model_2 weight summary
+----------------+-------+----------------+
|      name      | index |     value      |
+----------------+-------+----------------+
|  (intercept)   |  None | -54410676.1152 |
|  sqft_living   |  None | 304.449298057  |
|    bedrooms    |  None | -116366.043231 |
|   bathrooms    |  None | -77972.3305135 |
|      lat       |  None | 625433.834953  |
|      long      |  None | -203958.60296  |
| bed_bath_rooms |  None | 26961.6249092  |
+----------------+-------+----------------+
[7 rows x 3 columns]

model_3 weight 

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [35]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_1_RSS_train = get_residual_sum_of_squares(model_1, train_data, train_data['price'])
print "model_1 RSS", model_1_RSS_train

model_2_RSS_train = get_residual_sum_of_squares(model_2, train_data, train_data['price'])
print "model_2 RSS", model_2_RSS_train

model_3_RSS_train = get_residual_sum_of_squares(model_3, train_data, train_data['price'])
print "model_3 RSS", model_3_RSS_train

model_1 RSS 9.71328233544e+14
model_2 RSS 9.61592067856e+14
model_3 RSS 9.05276314555e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [36]:
# Compute the RSS on TESTING data for each of the three models and record the values:
model_1_RSS_test = get_residual_sum_of_squares(model_1, test_data, test_data['price'])
print "model_1 test RSS", model_1_RSS_test

model_2_RSS_test = get_residual_sum_of_squares(model_2, test_data, test_data['price'])
print "model_2 test RSS", model_2_RSS_test

model_3_RSS_test = get_residual_sum_of_squares(model_3, test_data, test_data['price'])
print "model_3 test RSS", model_3_RSS_test

model_1 test RSS 2.26568089093e+14
model_2 test RSS 2.24368799994e+14
model_3 test RSS 2.51829318952e+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected?Think about the features that were added to each model from the previous.